In [1]:
import pandas as pd
import numpy as np
from pyemd import emd_samples
import plotly.graph_objects as go
from IPython.display import display

# To import packages from the parent directory
import sys
sys.path.insert(0,'..')

# Autoreload allows us to update our code mid-notebook
%load_ext autoreload
%autoreload 2

# Our code
from main import *

In [7]:
file_path = '../data/20200423-1804_states.pickle'
#file_path = '../data/sample_morphologies.pickle'
df = pd.read_pickle(file_path)
print(f'Length of df: {len(df)}')

Length of df: 11681


In [8]:
sample = df.iloc[40]

sample_image = np.array(sample.image).reshape((100, 400))

fig = visualization.get_image_figure(sample_image)

print(sample)
display(fig)

BR                                                       0.53
CHI                                                         4
count                                                     300
image       [0.972673773765564, 0.6607276797294617, 0.0733...
timestep                                                   40
version                                                     2
Name: 40, dtype: object


FigureWidget({
    'data': [{'hoverinfo': 'none',
              'showscale': False,
              'type': 'hea…

In [25]:
traj_data = df[
    (df.BR  == 0.5) &
    (df.CHI == 2.2) &
    (df.version == 0)
]

In [26]:
traj = Trajectory(br=0.5, chi=2.2, version=0)

In [29]:
traj.morphologies

[]

In [32]:
traj.cache_morphologies(images=traj_data.image, timesteps=traj_data.timestep)

In [33]:
traj.morphologies[-1].viz()

FigureWidget({
    'data': [{'hoverinfo': 'none',
              'showscale': False,
              'type': 'hea…

## Distance Matrix

In [34]:
chi_values = np.sort(df.CHI.unique())
br_values = np.sort(df.BR.unique())
version_values = np.sort(df.version.unique())

print(f'chi_values     : {chi_values}')
print(f'br_values      : {br_values}')
print(f'version_values : {version_values}')

chi_values     : [2.2 2.8 3.4 4. ]
br_values      : [0.5  0.53 0.56 0.59]
version_values : [0 1 2 3]


In [55]:
print("Loading Traj 1")
traj_1 = load_trajectory_from_dataframe(
    df[
        (df.BR  == 0.5) &
        (df.CHI == 2.2) &
        (df.version == 0)
    ]
)

print("Loading Traj 2")
traj_2 = load_trajectory_from_dataframe(
    df[
        (df.BR  == 0.5) &
        (df.CHI == 2.2) &
        (df.version == 1)
    ]
)

print("Loading Traj 3")
traj_3 = load_trajectory_from_dataframe(
    df[
        (df.BR  == 0.53) &
        (df.CHI == 2.8) &
        (df.version == 0)
    ]
)

Loading Traj 1


NameError: name 'df' is not defined

In [16]:
traj_1 = Trajectory(br=0.5, chi=2.2, version=0)

In [10]:
traj_1.cache_signatures(perimeter_area_ratio_sig)

In [17]:
traj_1.morphologies[0].signatures

{'perimeter_area_ratio_sig': array([0.84488765, 0.74167634, 1.13807119, 0.6       , 0.88672954,
        0.        , 0.9279188 , 1.02071068, 0.94222159, 0.        ,
        0.        , 0.82071068, 0.33333333, 0.        , 0.77693089,
        0.86982524, 0.        , 0.88284271, 0.40236893, 0.5       ,
        0.8017767 , 1.09204746, 0.90236893, 1.0345178 , 0.73138298,
        1.04142136, 0.8       , 0.76654368, 0.83295593, 0.74913058,
        0.        , 0.        , 0.81903559, 0.        , 0.82998316,
        0.        , 0.91190075, 1.        , 0.        , 0.        ,
        1.13807119, 0.84142136, 0.93688672, 0.6       , 0.84142136,
        0.84142136, 0.75192075, 0.33333333, 0.33333333, 0.        ,
        0.73372815, 0.        , 0.        , 0.82761424, 0.        ,
        0.        , 0.75575565, 0.87313708, 0.        , 0.91190075,
        0.33333333, 0.88284271, 0.40236893, 0.72189514, 0.        ,
        1.0517767 , 0.77032018, 0.        , 0.69767645, 0.        ,
        1.10355339, 

In [42]:
def get_dist_matrix(traj_x, traj_y, sig_func):
    dist_matrix = []

    for morph_x in traj_x.morphologies:
        row = []
        sig_x = apply_to_components(morph_x.components, sig_func)
        for morph_y in traj_y.morphologies:
            sig_y = apply_to_components(morph_y.components, sig_func)
            
            if len(sig_x) == 0:
                sig_x = [0.]
            if len(sig_y) == 0:
                sig_y = [0.]
            row.append(emd_samples(sig_x, sig_y))
        dist_matrix.append(row)

    return np.array(dist_matrix)

In [52]:
perimeter_area_ratio_sig.__name__

'perimeter_area_ratio_sig'

In [45]:
dist_matrix = get_dist_matrix(traj_1, traj_2, perimeter_area_ratio_sig)

In [46]:
widget = visualization.get_distance_matrix_widget(
    dist_matrix,
    traj_1,
    traj_2
)
display(widget)

    'data': [{'type': 'heatmap',
              'uid': '00bbcf92-02c2-4c92-b729-d…

In [49]:
dist_matrix = get_dist_matrix(traj_1, traj_3, perimeter_area_ratio_sig)

In [51]:
widget = visualization.get_distance_matrix_widget(
    dist_matrix,
    traj_1,
    traj_3
)
display(widget)

    'data': [{'type': 'heatmap',
              'uid': 'fe370d5a-0212-4e90-801e-1…

# MDS

In [ ]:
from sklearn.manifold import MDS
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt

In [ ]:
dist_matrix = []

sort_by = ['CHI', 'BR']
for _, x_row in df_sigs.sort_values(sort_by).iterrows():
    matrix_row = []
    for _, y_row in df_sigs.sort_values(sort_by).iterrows():
        matrix_row.append(emd_samples(x_row.perimeter_area_ratio_sig, y_row.perimeter_area_ratio_sig))
    dist_matrix.append(matrix_row)

dist_matrix = np.array(dist_matrix)

In [ ]:
plt.imshow(dist_matrix)

In [ ]:
dist_mds = MDS(dissimilarity='precomputed')
dist_embedding = dist_mds.fit_transform(dist_matrix)

dist_embedding[:, 0] -= min(dist_embedding[:, 0])
dist_embedding[:, 1] -= min(dist_embedding[:, 1])

dist_embedding[:, 0] /= max(dist_embedding[:, 0])
dist_embedding[:, 1] /= max(dist_embedding[:, 1])

plt.scatter(dist_embedding[:, 0], dist_embedding[:, 1], c=np.linspace(0, 1, num=dist_embedding.shape[0]))
plt.show()

In [ ]:
print(br_values)
print(chi_values)

In [ ]:
keys = [
    { 'br': 0.5, 'chi': 2.8, 'version': 0 },
    { 'br': 0.5, 'chi': 2.8, 'version': 1 },
    { 'br': 0.56, 'chi': 3.4, 'version': 0 },
    { 'br': 0.56, 'chi': 3.4, 'version': 1 }
]

sigs = []
colors = []
color_options = ['r', 'b', 'k', 'c']
for kdx, key in enumerate(keys):
    color_option = color_options[kdx]
    traj = df[(df.BR == key['br']) & (df.CHI == key['chi']) & (df.version == key['version']) & (df.timestep != 0)]
    for idx, row in traj.iterrows():
        image = row.image.reshape((100, 400))
        comps = processing.extract_components(image, background=1)
        sig = processing.apply_to_components(comps, processing.perimeter_area_ratio_sig)
        if len(sig) == 0:
            continue
        sigs.append(sig)
        colors.append(color_option)

In [ ]:
dist_matrix = []

for sig_x in sigs:
    matrix_row = []
    for sig_y in sigs:
        matrix_row.append(emd_samples(sig_x, sig_y))
    dist_matrix.append(matrix_row)

dist_matrix = np.array(dist_matrix)

In [ ]:
dist_mds = MDS(dissimilarity='precomputed')
dist_embedding = dist_mds.fit_transform(dist_matrix)

# dist_embedding[:, 0] -= min(dist_embedding[:, 0])
# dist_embedding[:, 1] -= min(dist_embedding[:, 1])

# dist_embedding[:, 0] /= max(dist_embedding[:, 0])
# dist_embedding[:, 1] /= max(dist_embedding[:, 1])

plt.scatter(dist_embedding[:, 0], dist_embedding[:, 1], c=colors)
plt.show()

In [ ]:
plt.imshow(dist_matrix)